Notebook to prototype injestion of NYC Building Shape Data, isertion into DuckDB, creating spatial index, and running simple GIS query.


In [3]:
import duckdb
import polars as pl
from sodapy import Socrata
import requests
import os

In [2]:
#SOCRATA SETUP

# Load API key from file
SOCRATA_API_KEY = os.getenv('SOCRATA_API_KEY')

# source domain for NYC Open Data on Socrata
socrata_domain = 'data.cityofnewyork.us'

# initialize client
client = Socrata(
    socrata_domain,
    app_token=SOCRATA_API_KEY,  # Changed from None to api_key
    timeout=100
)

# examine object
print(client)

In [3]:

nyc_shape_id = '5zhs-2jue'

In [7]:

def query_data_nycod(dataset_id, custom_limit=None, columns=None, where=None):
    """
    Fetch data from NYC Open Data using sodapy client.

    Parameters
    ----------
    dataset_id : str
        The dataset ID (e.g., '5zhs-2jue').
    custom_limit : int, optional
        Custom row limit. If None, fetches entire dataset (paginated).
    columns : list of str or str, optional
        Specific columns to fetch. None or '*' fetches all.
    where : str, optional
        SoQL WHERE clause (without the 'WHERE' keyword).
        Example: "bin LIKE '5%'" or "bin IN ('1001', '1002')"

    Returns
    -------
    pl.DataFrame
    """

    # --- Get total dataset row count (unfiltered) ---
    total_count_result = client.get(dataset_id, select='COUNT(*)')
    total_dataset_rows = int(list(total_count_result[0].values())[0])

    # --- Get filtered row count if a WHERE clause is used ---
    if where:
        filtered_count_result = client.get(dataset_id, select='COUNT(*)', where=where)
        filtered_rows = int(list(filtered_count_result[0].values())[0])
        print(f"Total rows in dataset: {total_dataset_rows:,}")
        print(f"Rows matching filter:  {filtered_rows:,}")
        target_rows = filtered_rows
    else:
        print(f"Total rows in dataset: {total_dataset_rows:,}")
        target_rows = total_dataset_rows

    # --- Determine effective limit ---
    if custom_limit is not None:
        limit = custom_limit
        if limit < target_rows:
            print(
                f"⚠️  custom_limit ({limit:,}) is less than matching rows "
                f"({target_rows:,}). Result will be truncated."
            )
    else:
        limit = target_rows

    # --- Build base query params ---
    query_params = {}
    if columns and columns != '*':
        query_params['select'] = ", ".join(columns) if isinstance(columns, list) else columns
    if where:
        query_params['where'] = where

    # --- Paginated fetch ---
    page_size = 50_000  # Socrata practical ceiling per request
    all_results = []
    offset = 0
    remaining = limit

    while remaining > 0:
        fetch_size = min(page_size, remaining)
        query_params['limit'] = fetch_size
        query_params['offset'] = offset

        page = client.get(dataset_id, **query_params)
        if not page:
            break
        all_results.extend(page)
        offset += len(page)
        remaining -= len(page)

        # If API returned fewer rows than requested, we've hit the end
        if len(page) < fetch_size:
            break

    rows_returned = len(all_results)
    print(f"Rows returned:         {rows_returned:,}")

    # --- Post-fetch warnings ---
    if rows_returned < target_rows and custom_limit is None:
        print(
            f"⚠️  Expected {target_rows:,} rows but only received "
            f"{rows_returned:,}. The API may have truncated results."
        )

    return pl.DataFrame(all_results)

In [10]:
df_bin5 = query_data_nycod(
    nyc_shape_id,
    where="bin >= 5000000 AND bin < 5001000"
)
df_bin5.head()

Total rows in dataset: 1,082,917
Rows matching filter:  929
Rows returned:         929


the_geom,bin,doitt_id,shape_area,base_bbl,objectid,construction_year,feature_code,geom_source,ground_elevation,height_roof,last_edited_date,last_status_type,mappluto_bbl,shape_length
struct[2],str,str,str,str,str,str,str,str,str,str,str,str,str,str
"{""MultiPolygon"",[[[[-74.075406, 40.638514], [-74.075418, 40.638513], … [-74.075406, 40.638514]]]]}","""5000001""","""486250""","""780.74609375""","""5000010010""","""378979""","""1884""","""2100""","""Photogrammetric""","""24""","""57.52""","""2017-08-22T19:36:03.000Z""","""Constructed""","""5000010010""","""128.46454922521497"""
"{""MultiPolygon"",[[[[-74.075438, 40.640754], [-74.075435, 40.64074], … [-74.075438, 40.640754]]]]}","""5000002""","""421488""","""684.1875""","""5000010051""","""114462""","""1923""","""2100""","""Photogrammetric""","""55""","""17.81""","""2019-03-01T21:02:45.000Z""","""Constructed""","""5000010051""","""139.35623062648912"""
"{""MultiPolygon"",[[[[-74.075201, 40.641263], [-74.07519, 40.64117], … [-74.075201, 40.641263]]]]}","""5000004""","""335705""","""2325.2890625""","""5000010055""","""307972""","""1932""","""2100""","""Photogrammetric""","""55""","""40.68""","""2017-08-22T18:19:12.000Z""","""Constructed""","""5000010055""","""200.65168947285508"""
"{""MultiPolygon"",[[[[-74.099352, 40.643318], [-74.099351, 40.64331], … [-74.099352, 40.643318]]]]}","""5000005""","""33053""","""160.00390625""","""5000700001""","""949952""","""1920""","""2100""","""Photogrammetric""","""51""","""35.5""","""2017-08-22T17:52:39.000Z""","""Constructed""","""5000700001""","""54.820091199183771"""
"{""MultiPolygon"",[[[[-74.073792, 40.637971], [-74.073889, 40.637961], … [-74.073792, 40.637971]]]]}","""5000007""","""696704""","""237.35546875""","""5000010220""","""842682""","""1910""","""2100""","""Photogrammetric""","""10""","""13.41""","""2017-08-22T18:26:22.000Z""","""Constructed""","""5000010220""","""65.417397366291965"""
